In [1]:
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate

In [2]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

## Elegir si enseñar o recordar (evita el uso de tokens en OpenAI)

### 1. Enseñar

In [3]:
# Carga de documentos y extracción de información
# (asegúrate de que haya PDFs en la carpeta documentos)
documents = DirectoryLoader('./documentos/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')
# Separador de texto (límita el texto para que sea soportado por el LLM)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
textos = text_splitter.split_documents(documents)
# Se instruye (creación de un knowledge-base) con la info
# proporcionada
persist_directory = input('¿Cómo se llama este knoledge-base?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=textos, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
# Se guarda el conocimiento generado en el directorio actual
# para ser usado sin tener que instruir al LLM
vectordb.persist()
vectordb = None
os.system(f'zip -r db.zip ./{persist_directory}')

c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


1

### 2. Recordar (si y solo si Enseñar=Done)

In [4]:
# Now we can load the persisted database from disk, and use it as normal. 
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
#retriever = vectordb.as_retriever()

In [5]:
# Se especifica el LLM de OpenAI
turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)
# Se personaliza el LLM
template = """
Eres Prometeo, un asistente personal de revisión biliográfica que habla Español.

Tu tarea consiste en:

1. Leer detalladamente la información proporcionada en documentos que generalmente son
artículos científicos en formato PDF.

2. Proporcionar respuestas extremadamente detalladas a cualquier tipo de pregunta relacionada 
con el contexto, especialmente a preguntas específicas. Evita concluir tus respuestas con un 
párrafo que comience con 'En resumen,...'. Varía las conclusiones de tus respuestas para que 
sean más diversas y creativas.

3. Ser carismático y ofrecer información sobre ti y tus funciones.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables específicamente del contexto.

NUNCA terminar las respuestas con un resumen o una conclusión predecible.

Teniendo en cuenta TODO lo anterior, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
# Creación de cadena
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": prometeo_prompt},
                                       return_source_documents=True)
# Formato y referencias de las respuestas
def wrap_text_preserve_newlines(text, width=70):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nReferencias:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [6]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Cuáles son los principales resultados de la investigación?


c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Los principales resultados de la investigación revelan que la
producción de trabajos sobre innovación abierta en pymes ha tenido un
aumento significativo en los últimos años, destacando el año 2018 con
63 trabajos y el año 2019 con 55 trabajos. Se identificó que la
mayoría de los documentos son artículos académicos, seguidos de
documentos de sesiones, memorias y capítulos de libros. Además, se
encontró que los países con mayor contribución de artículos son
Italia, Reino Unido, Francia, Corea del Sur, China y España. En cuanto
a los autores, se identificaron cuatro clústeres con diferentes
enfoques de estudio, abarcando desde la influencia de la cooperación
en el desempeño de las pymes hasta el análisis de cómo las redes
sociales y el uso de datos abiertos contribuyen a la innovación
abierta. Asimismo, se destaca la importancia de realizar estudios
cuantitativos con muestras más grandes de empresas para formular
hipótesis y crear herramientas orientadas a la gestión de la
innovación abi

In [7]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

{'query': '¿Cuáles son los principales resultados de la investigación?',
 'result': 'Los principales resultados de la investigación muestran que la producción de trabajos sobre innovación abierta en pymes ha ido en aumento en los últimos años, con un pico en el año 2018. Se observa que la mayoría de los documentos son artículos de investigación, seguidos de documentos de sesiones o memorias y capítulos de libros. En cuanto a las fuentes donde se encontraron la mayor cantidad de documentos, se destaca el libro "Researching open innovation in SMEs", seguido de la revista "Open Innovation and Knowledge Management in Small and Medium Enterprises". Además, se identificaron los países con mayor contribución de artículos, siendo Italia y el Reino Unido los más destacados. En cuanto a los autores, se agruparon en clústeres y se destacaron trabajos que analizan la influencia de la cooperación en el desempeño de las pymes, así como el impacto de las redes sociales en la innovación abierta. En re

In [8]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)


¿Cuál es la relación entre las políticas públicas, la innovación
abierta y la internacionalización en las pymes según los autores
mencionados en el documento proporcionado?

Referencias:
documentos\2145-941X-pege-50-246.pdf
documentos\2145-941X-pege-50-246.pdf
documentos\2145-941X-pege-50-246.pdf
documentos\2145-941X-pege-50-246.pdf
documentos\2145-941X-pege-50-246.pdf


In [9]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

{'query': '',
 'result': '¿Cuál es el impacto de la innovación abierta en las pymes agroindustriales en Colombia?',
 'source_documents': [Document(page_content='272 pensamiento & gestión, 50. Universidad del Norte, 246-273, 2021Rubén Darío Echeverri Romero, Leidi Ruano-Arcos, Sara Isabel Bolaños DelgadoRuano -Arcos, L. D., Echeverri Romero, R. D., Rodríguez Orejuela, H. A.,  Castellanos, T. F. S. y Pineda Ospina, D. L. (2016). Public Policy for  the promotion of innovation in the Food Sector in Colombia. Cuader - nos de Administración , 32(56), 100-114. http://www.scielo.org.co/scielo. php?pid=S0120-46452016000200100&script=sci_abstract&tlng=en Ruiz, J. L., Polo Otero, J. L., Arrieta Barcasnegras, A. A. y Vélez Durán, L. F.  (2018). Determinantes del grado de apertura de las pymes agroindustriales:  una aplicación para el departamento del Atlántico. Revista Desarrollo y Socie - dad, 80, 189-229. https://doi.org/10.13043/ DYS.80.6 Santoro, G., Ferraris, A. y Winteler, D. J. (2019). Open

In [10]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)


Eres Prometeo, un asistente personal de revisión biliográfica que habla Español.

Tu tarea consiste en:

1. Leer detalladamente la información proporcionada en documentos que generalmente son
artículos científicos en formato PDF.

2. Proporcionar respuestas extremadamente detalladas a cualquier tipo de pregunta relacionada 
con el contexto, especialmente a preguntas específicas. Evita concluir tus respuestas con un 
párrafo que comience con 'En resumen,...'. Varía las conclusiones de tus respuestas para que 
sean más diversas y creativas.

3. Ser carismático y ofrecer información sobre ti y tus funciones.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables específicamente del contexto.

NUNCA terminar las respuestas con un resumen o una conclusión predecible.

Teniendo en cuenta TODO lo anterior, responde la siguiente pregunta: {question}

